In [1]:
from bs4 import BeautifulSoup as BS
import requests
import pandas as pd
import re

In [2]:
df = pd.DataFrame()

URLS_1 = ['https://www.exchange-rates.org/converter/usd-eur', 'https://www.exchange-rates.org/converter/eur-usd', 
         'https://www.exchange-rates.org/converter/uah-usd', 'https://www.exchange-rates.org/converter/usd-uah', 
         'https://www.exchange-rates.org/converter/eur-uah', 'https://www.exchange-rates.org/converter/uah-eur']

for URL_1 in URLS_1:
    
    from_cur = URL_1[-7:].split('-')[0].upper()
    to_cur = URL_1[-7:].split('-')[1].upper()
    
    req_1 = requests.get(URL_1)
    soup = BS(req_1.text, "html.parser")

    orig_path = "https://www.exchange-rates.org"

    urls = [orig_path + i[:-6] for i in re.findall(r'/\S*\d', str(soup.find_all('ul', class_='rates-by-year')[0]))]

    df_for_one = pd.DataFrame()

    for url in urls:
        req_2 = requests.get(url)
        soup_2 = BS(req_2.text, "html.parser")
        dates = [pd.to_datetime(date.text) for date in soup_2.find_all('a', class_='n')]
        values = [float(val.text.split()[-2]) for val in soup_2.find_all('span', class_='n')]
        _ = pd.DataFrame(values, index=dates, columns=[f'1_{from_cur}_to_{to_cur}'])
        df_for_one = pd.concat([df_for_one,_], axis=0)
    
    df_for_one = df_for_one.sort_index()
    display(df_for_one.tail())
        
    df = pd.concat([df, df_for_one], axis=1)

,1_USD_to_EUR
2024-05-07,0.9300
2024-05-08,0.9302
2024-05-09,0.9273
2024-05-10,0.9278
2024-05-13,0.9268


,1_EUR_to_USD
2024-05-07,1.0753
2024-05-08,1.0750
2024-05-09,1.0784
2024-05-10,1.0779
2024-05-13,1.0790


,1_UAH_to_USD
2024-05-07,0.02544
2024-05-08,0.02537
2024-05-09,0.02536
2024-05-10,0.02527
2024-05-13,0.02525


,1_USD_to_UAH
2024-05-07,39.310
2024-05-08,39.414
2024-05-09,39.427
2024-05-10,39.578
2024-05-13,39.608


,1_EUR_to_UAH
2024-05-07,42.270
2024-05-08,42.370
2024-05-09,42.516
2024-05-10,42.661
2024-05-13,42.735


,1_UAH_to_EUR
2024-05-07,0.02366
2024-05-08,0.02360
2024-05-09,0.02352
2024-05-10,0.02344
2024-05-13,0.02340


In [3]:
df = df.dropna().sort_index()
df.index.name = 'date'
df

,1_USD_to_EUR,1_EUR_to_USD,1_UAH_to_USD,1_USD_to_UAH,1_EUR_to_UAH,1_UAH_to_EUR
date,,,,,,
2015-01-01,0.8265,1.2098,0.06322,15.819,19.138,0.05225
2015-01-02,0.8324,1.2014,0.06322,15.819,19.004,0.05262
2015-01-03,0.8335,1.1997,0.06322,15.819,18.978,0.05269
2015-01-04,0.8370,1.1946,0.06265,15.962,18.900,0.05243
2015-01-05,0.8391,1.1918,0.06322,15.818,18.852,0.05305
...,...,...,...,...,...,...
2024-05-07,0.9300,1.0753,0.02544,39.310,42.270,0.02366
2024-05-08,0.9302,1.0750,0.02537,39.414,42.370,0.02360
2024-05-09,0.9273,1.0784,0.02536,39.427,42.516,0.02352


In [4]:
df.to_csv('/diploma_info/datalake/currency_rates.csv')